In [32]:
import h5py
import scipy.io as io
import PIL as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter 
import scipy
from scipy import spatial
import json
from matplotlib import cm as CM
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Functions

In [21]:
def gaussian_filter_density(ground_truth):
    '''Generates a density map using Gaussian filter transformation.'''
    density = np.zeros(ground_truth.shape, dtype=np.float32)
    ground_truth_count = np.count_nonzero(ground_truth)
    
    if ground_truth_count == 0:
        return density

    # Find out the K nearest neighbours using a KDTree
    index_of_nonzero_elements = np.nonzero(ground_truth)
    points = np.array(list(zip(index_of_nonzero_elements[1].ravel(), index_of_nonzero_elements[0].ravel())))
    leafsize = 2048
    
    # build kdtree
    tree = spatial.KDTree(points.copy(), leafsize=leafsize)
    # query kdtree
    distances, _ = tree.query(points, k=4)
        
    for i, point in enumerate(points):
        point_2d = np.zeros(ground_truth.shape, dtype=np.float32)
        point_2d[point[1],point[0]] = 1.
        if ground_truth_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(ground_truth.shape))/2./2. #case: 1 point
        
        #Convolve with the gaussian filter
        density += gaussian_filter(point_2d, sigma, mode='constant')
    return density


def create_density_maps(img_paths):
    """For each path given, loads the corresponding image and its ground truth. 
    The ground truth can be seen as the x,y coordinates of the detected personds. 
    Out of this ground truth a hot encoded matrix is generated for the image. 
    This is a matrix with same shape as the image, and whose all values are 0 except
    for those where a person is found (with value 1). From this matrix the density 
    map is constructed via gaussian transformations in another function and saved in h5 file. """
    for img_path in tqdm(img_paths):
        # Load sparse matrix
        mat_file = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
        
        #Read image
        img= plt.imread(img_path)
        # Create a zero matrix of image size
        k = np.zeros((img.shape[0],img.shape[1]))
        
        ground_truth = mat_file["image_info"][0,0][0,0][0]

        #Generate hot encoded matrix of sparse matrix
        for i in range(len(ground_truth)):
            if int(ground_truth[i][1])<img.shape[0] and int(ground_truth[i][0])<img.shape[1]:
                k[int(ground_truth[i][1]), int(ground_truth[i][0])]=1
        
        # generate density map
        k = gaussian_filter_density(k)
        
        # File path to save density map
        file_path = img_path.replace('.jpg','.h5').replace('images','density_maps')
        # Create directory if it doesn't exist
        if not os.path.exists(os.path.dirname(file_path)):
            os.makedirs(os.path.dirname(file_path))

        with h5py.File(file_path, 'w') as hf:
                hf['density'] = k

# Load the data

In [22]:
root = 'data\shangai_dataset'
part_A_train = os.path.join(root,'part_A\\train_data','images')
part_A_test = os.path.join(root,'part_A\\test_data','images')
part_B_train = os.path.join(root,'part_B\\train_data','images')
part_B_test = os.path.join(root,'part_B\\test_data','images')
path_sets = [part_A_train, part_A_test, part_B_train, part_B_test]

In [23]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
print(f'Number of images: {len(img_paths)}')
print(f'Example of image path: {img_paths[0]}')

Number of images: 1198
Example of image path: data\shangai_dataset\part_A\train_data\images\IMG_1.jpg


# Create density maps

In [24]:
create_density_maps(img_paths)

100%|██████████| 1198/1198 [3:31:44<00:00, 10.61s/it] 


Let's visualize the density map of an image.

In [ ]:
# Image
img_22 = Image.open(img_paths[22])
plt.imshow(img_22)

In [19]:
# Its density map
file_path_22 = img_paths[22].replace('.jpg','.h5').replace('images','ground') 
ground_truth_file_22 = h5py.File(file_path_22,'r')
groundtruth_22 = np.asarray(ground_truth_file_22['density'])
plt.imshow(groundtruth_22,cmap=CM.jet)
print("Sum = " ,np.sum(groundtruth_22))

data\shangai_dataset\part_A\train_data\ground\IMG_119.h5


## Mall dataset


In [25]:
root_mall = 'data\mall_dataset'
images_mall = np.load('data\mall_dataset\images.npy')
labels_mall = np.load('data\mall_dataset\labels.npy')

In [30]:
print(f'Image array shape: {images_mall.shape}')
print(f'Labels array shape: {labels_mall.shape}')

Image array shape: (2000, 480, 640, 3)
Labels array shape: (2000, 1)


Let's do a train-test split:

In [33]:
X_train_mall, X_test_mall, y_train_mall, y_test_mall = train_test_split(images_mall, labels_mall, test_size=0.2, random_state=42)

# print the shapes of the train and test sets
print('X_train shape:', X_train_mall.shape)
print('y_train shape:', y_train_mall.shape)
print('X_test shape:', X_test_mall.shape)
print('y_test shape:', y_test_mall.shape)

X_train shape: (1600, 480, 640, 3)
y_train shape: (1600, 1)
X_test shape: (400, 480, 640, 3)
y_test shape: (400, 1)
